In [1]:
import logging

import probtorch
import torch

import combinators
import hmm
import importance
import mcmc
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [3]:
init_hmm = combinators.HyperPopulation(hmm.init_hmm, (1,), hyper=hmm_params)
hmm_step = combinators.PrimitiveCall(hmm.hmm_step)
hmm_run = combinators.Reduce(hmm_step, generator=lambda: range(50), initializer=init_hmm)

In [4]:
(z_last, mu, sigma, pi, pi0), generative = hmm_run()

In [5]:
data = {k.rpartition('/')[-1]: rv.value for (k, rv) in generative.filter(lambda k, rv: 'X_' in k)}

In [8]:
num_particles = 250

In [9]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    smc_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [10]:
init_hmm = combinators.HyperPopulation(hmm.init_hmm, , trainable=smc_hmm_params)

In [11]:
smc_hmm = importance.smc(hmm_step, (num_particles,), range(50), initializer=init_hmm)

In [12]:
hmm_proposal = combinators.Reduce.sequence(combinators.Model(hmm.hmm_step), 50, initializer=init_hmm)

In [13]:
particle_mh = mcmc.IndependentMH(smc_hmm, hmm_proposal, num_particles*10)

In [14]:
samples, elbos = particle_mh(trace=importance.ResamplerTrace(num_particles, data=data))
inference = particle_mh.trace

In [15]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inference[key].value == generative[key].value).sum().to(dtype=torch.float) / inference.num_particles
    print('SMC accuracy at time %d: %f' % (t, accuracy))

SMC accuracy at time 1: 0.680000
SMC accuracy at time 2: 1.000000
SMC accuracy at time 3: 0.000000
SMC accuracy at time 4: 0.000000
SMC accuracy at time 5: 0.000000
SMC accuracy at time 6: 0.320000
SMC accuracy at time 7: 0.364000
SMC accuracy at time 8: 0.680000
SMC accuracy at time 9: 0.636000
SMC accuracy at time 10: 0.364000
SMC accuracy at time 11: 0.684000
SMC accuracy at time 12: 0.000000
SMC accuracy at time 13: 0.508000
SMC accuracy at time 14: 0.104000
SMC accuracy at time 15: 0.424000
SMC accuracy at time 16: 0.636000
SMC accuracy at time 17: 0.432000
SMC accuracy at time 18: 0.316000
SMC accuracy at time 19: 0.428000
SMC accuracy at time 20: 0.256000
SMC accuracy at time 21: 0.636000
SMC accuracy at time 22: 0.312000
SMC accuracy at time 23: 0.468000
SMC accuracy at time 24: 0.744000
SMC accuracy at time 25: 0.360000
SMC accuracy at time 26: 0.896000
SMC accuracy at time 27: 0.256000
SMC accuracy at time 28: 0.216000
SMC accuracy at time 29: 1.000000
SMC accuracy at time 30

In [16]:
elbos

tensor([-137.0664, -137.0664, -137.0664,  ..., -134.6189, -134.6189,
        -134.6189], grad_fn=<CopySlices>)